## Import and data structuring

In [ ]:
import os
import sqlite3
import pandas as pd

# ## Note: The code currently assumes that the database file is called "AntennaPodBackup.db" and is one directory above the cloned github folder.
# con = sqlite3.connect(os.path.join((os.path.dirname(os.getcwd())),"AntennaPodBackup-2022-01-23.db"))

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from tkinter import *

# create the root window
root = tk.Tk()
root.title('Tkinter Open File Dialog')
root.resizable(False, False)
root.geometry('300x150')


def select_file():
    filetypes = (
        ('database files', '*.db'),
        ('All files', '*.*')
    )

    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='/',
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=filename
    )


# open button
open_button = ttk.Button(
    root,
    text='Open a File',
    command=select_file
)

open_button.pack(expand=True)


# run the application
root.mainloop()

con = sqlite3.connect(os.path.join(filename))

In [ ]:
# Read tables
FeedMedia = pd.read_sql_query("SELECT * from FeedMedia", con)
FeedItems = pd.read_sql_query("SELECT * from FeedItems", con)
Feeds = pd.read_sql_query("SELECT * from Feeds", con)

In [ ]:
# convert timestamp in ms to datetime objects (why are they in milliseconds?!)
FeedMedia.last_played_time = pd.to_datetime(FeedMedia.last_played_time, unit='ms')
FeedMedia.playback_completion_date = pd.to_datetime(FeedMedia.playback_completion_date, unit='ms')
FeedItems.pubDate = pd.to_datetime(FeedItems.pubDate, unit='ms')

In [ ]:
# for future analysis it might be useful to have the years, months and days ready
# FeedMedia['lpt_year']= FeedMedia['last_played_time'].dt.year
# FeedMedia['lpt_month']= FeedMedia['last_played_time'].dt.month
# FeedMedia['lpt_day']= FeedMedia['last_played_time'].dt.day
# FeedMedia['lpt_weekday']= FeedMedia['last_played_time'].dt.dayofweek
# FeedMedia['duration']= FeedMedia['duration']/1000/60
FeedMedia['percentage_played']= FeedMedia['position']/FeedMedia['duration']
# FeedMedia["duration"] = pd.to_timedelta(FeedMedia["duration"],unit="ms")
# FeedMedia['pcd_year']= FeedMedia['playback_completion_date'].dt.year
# FeedMedia['pcd_month']= FeedMedia['playback_completion_date'].dt.month
# FeedMedia['pcd_day']= FeedMedia['playback_completion_date'].dt.day

df = pd.merge(FeedMedia, FeedItems, left_on='feeditem', right_on='id', suffixes=('_media', '_item'))
df = pd.merge(df, Feeds, left_on='feed', right_on='id', suffixes=('', '_feed'))

df.percentage_played.fillna(0,inplace=True)

df["pub_year"] = df["pubDate"].dt.year

## Dash part

In [ ]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div(children = [
    dcc.Dropdown(
        id="podcasts", 
        placeholder="Select some podcasts", 
        clearable=False, 
        options=[{"label": y, "value": y} for y in df["title_feed"].unique()],
        multi=True
    ),
    dcc.RangeSlider(
        id='year_slider',
        min=1990,
        max=2030,
        step=1,
        value=[2018, 2022],
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    dcc.Graph(id="release", figure={}),
              
    dcc.Graph(id="bar-plot", figure={}),
    ])
    
    
@app.callback(Output("release", "figure"),
              Input("year_slider", "value"),
              Input("podcasts", "value"))
def year_range(year_slider,podcasts):
    # Scatter plot with weekday and duration in minutes
    df1 = df[df.playback_completion_date.dt.year.isin(range(year_slider[0],year_slider[1]))]
    if podcasts:
        df2 = df1[df1.title_feed.isin(podcasts)]
        fig = px.scatter(df2[df2["percentage_played"]>0], 
                 template="plotly_dark",
                 color_discrete_sequence=px.colors.qualitative.Bold,
                 y="duration", 
                 x="pubDate", 
                 color="title_feed", 
                 size = "percentage_played", 
                 height=500, 
                 range_x=year_slider,
                 )
    
    else:
        fig = px.scatter(df1[df1["percentage_played"]>0], 
                 template="plotly_dark",
                 color_discrete_sequence=px.colors.qualitative.Bold,
                 y="duration", 
                 x="pubDate", 
                 color="title_feed", 
                 size = "percentage_played", 
                 height=500, 
                 range_x=year_slider,
                 )
        
    return fig.update_layout(font_family="Rockwell",
                             showlegend=False,
                             legend=(dict(orientation="h", 
                                          title="", 
                                         )))

@app.callback(Output("bar-plot", "figure"),
              Input("year_slider", "value"),
              Input("podcasts", "value"))
def party(year_slider,podcasts):
    df1 = df[df.playback_completion_date.dt.year.isin(range(year_slider[0],year_slider[1]))]
    if podcasts:
        df2 = df1[df1.title_feed.isin(podcasts)]
        fig2 = px.bar(df2[df2["played_duration"]>0],
           template="plotly_dark",
           x=df2['playback_completion_date'].dt.dayofweek, 
           y=df2["played_duration"]/1000/60,
           color=df2['playback_completion_date'].dt.year.astype(str),
           color_discrete_sequence=px.colors.qualitative.Bold)
    
    else:
        fig2 = px.bar(df1[df1["played_duration"]>0],
           template="plotly_dark",
           x=df1['playback_completion_date'].dt.dayofweek, 
           y=df1["played_duration"]/1000/60,
           color=df1['playback_completion_date'].dt.year.astype(str),
           color_discrete_sequence=px.colors.qualitative.Bold)
        
    return fig2.update_layout(font_family="Rockwell",
                             showlegend=False,
                             legend=(dict(orientation="h", 
                                          title="", 
                                         )))
    
app.run_server(mode="jupyterlab")

In [ ]:
list(df.columns.values)

In [ ]:
# Scatter plot with weekday and duration in minutes
fig = px.scatter(df[df["lpt_year"] != 1970],
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 template="plotly_dark",
                 y="duration",
                 x="lpt_weekday", 
                 color="lpt_month", 
                 size="percentage_played", 
                 hover_name="title_feed",
                 hover_data={"title" : True,
                             "lpt_weekday" : False,
                             "lpt_month" : False,
                             "percentage_played" : False,
                             "duration" : True,},
                 title="Played Episodes",
                 height=500)
fig.update_layout(font_family="Rockwell",
                  legend=(dict(orientation="h", title="")))

In [ ]:
# Scatter plot with weekday and duration in minutes
fig = px.scatter(df, 
                 y="duration_minutes", 
                 x="pub_year", 
                 color="title_feed", 
                 size = "percentage_played", 
                 height=500, 
                 range_x=[2000,2025])
fig.update_layout(font_family="Rockwell",
                  legend=(dict(orientation="v", title="")))

In [ ]:
df

In [ ]:
df3 = df[df["lpt_year"]>1970].groupby(by=["lpt_year",
                                          #"lpt_month"
                                         ]).sum()
px.line(df3,
           template="plotly_dark",
           y="duration_minutes",
           x="lpt_month",#df3.index,#.get_level_values(0), 
           height=500,
           color=df3.index)

In [ ]:
px.bar(df[df["played_duration"]>0],#.groupby(['playback_completion_date'].dt.dayofweek),
       x=df['playback_completion_date'].dt.dayofweek, 
       y=df["played_duration"]/1000/60
      )

In [ ]:
df["played_duration"]